# Notebook de métricas y test

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path

# sube de notebooks/ a la raíz del repo

sys.path.append(str(Path.cwd().parent))

import src.functions as fc

%matplotlib inline 
%load_ext autoreload
%autoreload 2

In [18]:
# Hacemos merge

df2_1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt")
df2_2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt")
df2 = pd.concat([df2_1, df2_2], axis=0, ignore_index=True)


In [19]:
fc.save_data_clean(df2, "df2_merged")

Guardado: /Users/juaruibel/Projects/BOOTCAMP/Unit 05 - EDA and Inferential Stats/project2-cx-julio-juanjo/data/cleaned/20260211_1638_df2_merged.csv


'ok'

In [20]:
df2 = fc.load_data_merge("20260211_1638_df2_merged.csv")

In [21]:
df2.head()

,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04


In [22]:
df2.shape

(755405, 5)

In [ ]:
# Usuarios únicos que llegan a confirm 

usuarios_confirm = df2[df2["process_step"] == "confirm"]

usuarios_confirm = (df2[df2["process_step"] == "confirm"][["client_id"]].drop_duplicates())

num_usuarios_confirm = usuarios_confirm.shape[0]

In [58]:
# Client id únicos

num_client_id_unique = df2["client_id"].nunique()

tasa_finalizacion = round((num_usuarios_confirm/num_client_id_unique)*100, 2)

tasa_finalizacion

67.53

# TASA DE ERROR

In [61]:
df2["date_time"] = pd.to_datetime(df2["date_time"])

df2["date_time"]

0        2017-04-17 15:27:07
1        2017-04-17 15:26:51
2        2017-04-17 15:19:22
3        2017-04-17 15:19:13
4        2017-04-17 15:18:04
                 ...        
755400   2017-05-24 18:46:10
755401   2017-05-24 18:45:29
755402   2017-05-24 18:44:51
755403   2017-05-24 18:44:34
755404   2017-06-03 12:17:09
Name: date_time, Length: 755405, dtype: datetime64[ns]

In [63]:
df2 = df2.sort_values(["visitor_id", "visit_id", "date_time"])

df2

,client_id,visitor_id,visit_id,process_step,date_time
444289,5867343,100011869_84756704576,406596333_52561114200_989487,start,2017-06-05 02:47:42
444288,5867343,100011869_84756704576,406596333_52561114200_989487,step_1,2017-06-05 02:47:47
444287,5867343,100011869_84756704576,406596333_52561114200_989487,step_2,2017-06-05 02:48:03
444286,5867343,100011869_84756704576,406596333_52561114200_989487,step_3,2017-06-05 02:50:18
444285,5867343,100011869_84756704576,406596333_52561114200_989487,start,2017-06-05 02:51:11
...,...,...,...,...,...
128792,7042750,999998529_89539814462,583327415_19513940564_488564,step_2,2017-04-12 11:55:05
128791,7042750,999998529_89539814462,583327415_19513940564_488564,step_3,2017-04-12 11:55:13
128790,7042750,999998529_89539814462,583327415_19513940564_488564,confirm,2017-04-12 12:04:47
128789,7042750,999998529_89539814462,583327415_19513940564_488564,start,2017-04-12 12:09:29


In [ ]:
# Fechas y diferencia

df2["next_time"] = df2.groupby(["client_id","visit_id"])["date_time"].shift(-1)
df2["next_step"] = df2.groupby(["client_id","visit_id"])["process_step"].shift(-1)

df2["duration"] = df2["next_time"] - df2["date_time"]

# Filtros por step

start_s1 = df2[(df2["process_step"] == "start") & (df2["next_step"] == "step_1")]
s1_s2 = df2[(df2["process_step"] == "step_1") & (df2["next_step"] == "step_2")]
s2_s3 = df2[(df2["process_step"] == "step_2") & (df2["next_step"] == "step_3")]
s3_confirm = df2[(df2["process_step"] == "step_3") & (df2["next_step"] == "confirm")]

# Medias

media_paso1 = start_s1["duration"].mean()
media_paso2 = s1_s2["duration"].mean()
media_paso3 = s2_s3["duration"].mean()
media_paso4 = s3_confirm["duration"].mean()

media_paso1, media_paso2, media_paso3, media_paso4


(Timedelta('0 days 00:00:32.047011243'),
 Timedelta('0 days 00:00:35.255924491'),
 Timedelta('0 days 00:01:28.492120289'),
 Timedelta('0 days 00:01:50.173717708'))

# TASA DE ERROR

In [90]:
error1 = df2[(df2["process_step"] == "step_1") & (df2["next_step"] == "start")].shape[0]
error2 = df2[(df2["process_step"] == "step_2") & (df2["next_step"] == "step_1")].shape[0]
error3 = df2[(df2["process_step"] == "step_3") & (df2["next_step"] == "step_2")].shape[0]

error1, error2, error3

(23589, 12176, 9430)